# Agent Supervisor Architecture Application
Applying a supervisor approach to ideas of social media sentiment analysis

In [5]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

True

---

## Gathering API keys

LLMs and LangChain API key set in environment variable, `langchain_tracing_v2` allows langsmith agent tracing, `langchain_project` determines organization in langsmith

In [6]:
os.environ.get("LANGCHAIN_TRACING_V2")
os.environ.get("LANGCHAIN_ENDPOINT")
os.environ.get('LANGCHAIN_API_KEY')
os.environ.get("LANGCHAIN_PROJECT")
# os.environ["OPENAI_API_KEY"] = "apikey"

'Experiments Master'

In [7]:
llm = Ollama(model="llama3")

In [4]:
llm.invoke("how can langsmith help with testing?")

"Langsmith can assist with testing in several ways:\n\n1. **Natural Language Processing (NLP) Tests**: Langsmith provides a range of NLP tests that can be used to evaluate the performance of language models, such as:\n\t* Tokenization: splitting text into individual words or tokens.\n\t* Sentiment Analysis: determining the emotional tone of text (positive, negative, neutral).\n\t* Entity Recognition: identifying named entities (people, organizations, locations).\n2. **Automated Testing**: Langsmith can automate testing for language-based applications, such as:\n\t* Chatbots: testing conversational flows and responses.\n\t* Virtual Assistants: evaluating their ability to understand voice commands.\n3. **Error Analysis**: Langsmith's NLP capabilities can help identify errors in text data, such as:\n\t* Spelling mistakes\n\t* Grammar errors\n\t* Misinterpreted meaning or context\n4. **Text Classification**: Langsmith can be used to test the performance of text classification models, such 

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation reader."),
    ("user", "{input}")
])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "What is the Add to 'Dataset functionality' in LangSmith?"})

'Excellent question!\n\nIn LangSmith, the "Add to \'Dataset\'" functionality allows you to merge or combine multiple datasets into a single dataset. This is particularly useful when working with complex documents that require integrating data from multiple sources.\n\nWhen you select this option, LangSmith will automatically identify the relevant information from each dataset and merge it into a single dataset. The resulting dataset will include all the original information, as well as any new information that was added during the merging process.\n\nFor instance, imagine you\'re working on a technical manual for a complex device that requires integrating data from multiple sources, such as user manuals, design specifications, and test results. By using the "Add to \'Dataset\'" functionality in LangSmith, you can easily combine these datasets into a single, comprehensive dataset that includes all the relevant information.\n\nThis feature is particularly useful when:\n\n1. Merging datas

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

embeddings = OllamaEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Based on the provided context, LangSmith can let you "visualize" test results.'

In [7]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user",
     "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Logging**: Every playground run is logged in the system and can be used to create test cases or compare with other runs.
2. **Initial Test Set**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
3. **Comparison View**: When prototyping different versions of your applications and making changes, it's important to see whether or not you've regressed with respect to your initial test cases. LangSmith provides a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of your application.
4. **Evaluation/Testing System**: Langsmith enables users to add runs as examples to datasets (from both the project page and within an annotation queue), expanding their test coverage on real-world scenarios.

Overall, LangSmith provides a platform for LLM appl

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is fa

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation betwe